# Simulated Annealing for Mars Lander Episode-1 

### Import modules and Utilities

In [1]:
from typing import List, Optional, Callable, Tuple, Dict, Union  # OneNote2021 #_python_ #_typing_
GENE = List[int]  # (rotate (-15 ~ +15),power (-1, 0, +1)) 避免無用的值域，下給 Web site 的 command 要轉換成 (rotate(-90 ~ +90), power(0,1,2,3,4))
GENOME = List[GENE]  # Genome 是一個 list 裡面都是 int
STATE = Dict[str,Union[int,float]]  # x,y,hspeed,power,.. etc 

import numpy as np
import re, math, random, pdb, json, copy
import peforth
peforth.bps = [i for i in range(100)]  # 預設有這麼多 Breakpoint ID
def bp(id=None,locals=None):
    # Breakpoint ID 不能超過 peforth.bps 保留，超過的無效。
    if id==None: 
        id = 0
        prompt='bp> '
    else:
        prompt="{}>".format(id)
    if id in peforth.bps: peforth.push(locals).ok(prompt, cmd="to _locals_")
peforth.bp = bp
peforth.push(bp).dictate("py: vm.bp=pop()")  # [ ] 忘了 vm.bp 有何用途？
peforth.dictate('''
    \ ------ breakpoint ------------------------------------------------------------
    : bp ." Usage: peforth.bp(11,locals())  # drop a breakpoint with ID=11" cr ;
    : bl // ( -- ) List all breakpoints
        __main__ :> peforth.bps 
        <py>
        bps = pop()
        print('Disabled breakpoints:')
        for i in range(len(bps)):
            if not bps[i]: 
                print(i, end=' ')
        print(); print('Enabled breakpoints:')
        count = 0
        for i in range(len(bps)):
            if bps[i]: 
                print(i, end=' ')
                count += 1
        print(); print('Enabled breakpoints count: {}/{}'.format(count,len(bps)))
        </py> cr ;
        /// Breakpoint commands:
        ///	  bl  - list all breakpoints (capital BL is white space) 
        ///	  be  - enable breakpoints, e.g. be 1 2 3 
        ///	  bd  - disable breakpoints, e.g. bd 1 2 3 
        ///	  be* - enable all breakpoints
        ///	  bd* - disable all breakpoints 
        /// Also: 
        ///   for i in [11,22,33]: peforth.bps[i]=0	 # disable breakpoints 11,22,33 
        ///   for i in [11,22,33]: peforth.bps[i]=i	 # enable  breakpoints 11,22,33 
    
    : bd // ( <1 2 3 4...> -- ) Disable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps )
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = 0
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : be // ( <1 2 3 4...> -- ) Enable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps ) 
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = i
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : bd* // ( -- ) Disable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = 0
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    
    : be* // ( -- ) Enable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = i
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    \ ------ xtack ------------------------------------------------------------
    [] value xstack xstack py: vm.xstack=pop() // ( -- array ) The xstack. 掛進 vm 就可以直接以 py> xstack 取用。
    : x@ xstack :> [-1] ; // ( -- n ) Get TOS of the xstack
    : x> xstack :> pop() ; // ( -- n ) Pop the xstack
    : >x xstack :: append(pop()) ; // ( n -- ) Push n into the xstack
    : .sx xstack . ; // ( -- ) List xstack 
    : xdrop x> drop ; // ( X: ... a -- X: ... ) drop xstack 
    : xdropall [] to xstack ; // ( X: ... -- X: empty ) clear xstack 
    ''')
%f version ==>


reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

version ==>
 1.26 (<class 'str'>)


In [2]:
# 把 genome 轉成 Web 接受的 command list 
def genome_to_command_list(genome:GENOME) -> List[List[int]] :
    p = 0  # initial power is always 0 
    a = [] # result command array of [rotate, thrust] paris 
    for i in genome :
        if i[1] == -1 :
            p = max(p-1, 0) 
            a.append([0, p])
            pass
        elif i[1] == 0 :
            a.append([0, p])
            pass
        elif i [1] == 1 :
            p = min(p+1, 4) 
            a.append([0, p])
            pass
        else:
            pass
    return a

%f genome_to_command_list :> ([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1]]) -->

genome_to_command_list :> ([[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1]]) --> [[0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 4], [0, 3], [0, 2], [0, 1], [0, 0], [0, 0]] (<class 'list'>)


# Episode-1 

In [3]:
# 最基本的定義 

gravity = -3.711        # constant on Mars
g = gravity             # 到了二維以上，重力需轉成向量

class State(STATE):

    def __init__(self,step=0,x=0,y=0,hspeed=0,
            vspeed=0,fuel=0,angle=0,power=0,force=0):
        self.step   = step
        self.x      = x
        self.y      = y
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.fuel   = fuel
        self.angle  = angle
        self.power  = power
        self.force  = force

    def next(self, angle:int, power:int):  
        # input angle,power 都是 offset 分別屬 -15~15 以及 -1,0,+1 
        assert power in [-1, 0, 1], 'expected input power is an offset [-1, 0, +1], given {}'.format(power)
        assert angle in range(-15,16), 'input angle range [-15:16], given {}'.format(angle)
        self.step   += 1
        self.angle  += angle
        self.angle   = -90 if self.angle < -90 else self.angle
        self.angle   =  90 if self.angle > 90 else self.angle
        self.power  += power
        self.power   = 0 if self.power < 0 else self.power
        self.power   = 4 if self.power > 4 else self.power
        self.fuel   -= self.power
        self.x       = self.x       # episode-1 不變
        self.hspeed  = self.hspeed  # episode-1 不變
        self.force   = g + self.power
        self.y       = self.y + self.vspeed + (1/2)*self.force
        self.vspeed += self.force
        return self  # 傳回值只是方便測試，可以這樣寫： %f state :> next(0,3.711) -->

    def __str__(self):  
        return str(self.__dict__)

fuel0 = 500             # episode-1 
state0 = State(0,0,2500,0,0,fuel0,0,0,0)  # episode-1 的起點 state


In [4]:
# 建置 command lists 錄製好幾種狀況方便實驗

sample_genomes = {"自由落體":[],"安全降落":[], "飛向太空":[], "差點成功":[], "更成功":[]}  
state_list = {"自由落體":{},"安全降落":{}, "飛向太空":{}, "差點成功":{}, "更成功":{}}  

# 自由落體
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(36):
    state.next(0,0);
    sample_genomes["自由落體"].append([0,0])
state_list["自由落體"]=state  # state 是獨立的，不需要 deep copy 

# 安全降落
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(9):
    state.next(0,0);
    sample_genomes["安全降落"].append([0,0])
for i in range(9,91):
    state.next(0,1);
    sample_genomes["安全降落"].append([0,1])
state_list["安全降落"]=state

# 飛向太空
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(83):
    state.next(0,1);
    sample_genomes["飛向太空"].append([0,1])
state_list["飛向太空"]=state

# 更成功
state = State(**state0.__dict__)  # init for ecah genome 
for i in range(9):
    state.next(0,0);
    sample_genomes["更成功"].append([0,0])
for i in range(9,81):
    state.next(0,1);
    sample_genomes["更成功"].append([0,1])
for i in range(7):
    state.next(0,-1);
    sample_genomes["更成功"].append([0,-1])
state_list["更成功"]=state

%f sample_genomes :> ["自由落體"] genome_to_command_list :> (pop()) -->
%f sample_genomes :> ["安全降落"] genome_to_command_list :> (pop()) -->
%f sample_genomes :> ["飛向太空"] genome_to_command_list :> (pop()) -->
%f sample_genomes :> ["更成功"] genome_to_command_list :> (pop()) -->

%f state_list :> ["自由落體"] -->  
%f state_list :> ["安全降落"] -->  
%f state_list :> ["飛向太空"] -->  
%f state_list :> ["更成功"]   -->  


sample_genomes :> ["自由落體"] genome_to_command_list :> (pop()) --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]] (<class 'list'>)
sample_genomes :> ["安全降落"] genome_to_command_list :> (pop()) --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0,

In [5]:
# 改寫 for Mars Lander 乍看以為要頭痛一會兒，其實 Mars Lander 的 genome 很簡單，就是一串 (int,int) 
# pairs 而已。前面基礎討論有提到，以 SA 而言就是 mutate 某一個 gene 也就是 command pair 而已。但是考慮 Mars
# Lander 的限制條件，以 epidode-1 而言，command 的改變要做的決定是 -1 0 +1 其中之一才對，而非 0~4. 

# 長度的問題，
# 我想的是需要另一個 function 不夠時直接 copy 最後一個 gene 頂替即可。
# 會發現不夠的是 call fitness function 的人，那就是由它處理。

def get_random_neighbour(genome: GENOME) -> None :
    neighbour = genome[:]  # Deep copy 利用 [:] slice copy, 若用 list(state) 不行！

    idx = random.randint(0,len(genome)-1) # 選 1 個 gene 
    cmd = random.randint(-1, 1)  # -1, 0, +1 當中選 1 個 thrust command (episode-1)

    neighbour[idx][1] = cmd
    return neighbour

In [6]:
# ○ 成功著陸的，油料剩越多的越好。
# ○ 一個油料耗盡一個墜毀哪個好？墜毀時的速度減到+-40 以內需要多少油料？
# ○ 油料耗盡時的高度距離與速度以自由落體觸地，然後把速度調整到+-40 以內需要多少油料？
# ○ 根據我之前對 y=100 (flat ground 的海拔高度) 的研究 該 state 來自 log 的最後一 state. 
#   當 Simulating 時，該 state 就是 y 首次跨越 flat ground 高度觸地著陸之時。
# ○ cost 必大於 0。 Score 用油料剩餘量計算，由 fuel0 扣下來可到負值。換算成 cost 就是 
#      cost = fuel0 - score
#   核算，當 score 為 fuel0 時 cost 為 0 無誤； 當 score 為 -fuel0 時 cost 為 2fuel0
#   甚至更高，也無誤。 直接用這個數字，配合 SA 的 Temperature 設定要用 visualized 
#   graphs 來訂定 即可。

def cost_of_state(state: STATE) -> int:
    # state = State(**state_all_power.__dict__) 
    score = None  # 有可能是負的，先給個看得出異常的初值。

    # 先判斷是否觸地。若觸地進一步查看速度合格是否合格。若未觸地就是飛走了或油料耗盡。
    if round(state.y,0) < 100:  # 若僅 state.y <= 100 則 100.45 也會被錯當成已著陸。
        # 觸地了，進一步查看速度是否合格
        if round(state.vspeed,0) >= -40 : # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小
            # 安全降落
            score = state.fuel
        else:  # 墜毀
            # 模擬噴火把速度降到合格範圍內。所剩油料就是成績。
            while True:
                state.next(0,1)  # 有個調整的過程，不是直接填 4 ！
                if round(state.vspeed,0) >= -40:
                    score = state.fuel
                    break
    else:  # 還在空中，飛走了或油料耗盡。
        # 先模擬 free fall 觸地。
        while True: 
            state.next(0,-1)  # 半空中開始模擬 free fall 小心別又錯了，不是直接填 0 ！！
            if round(state.y,0) < 100: break
        # 然後噴火把速度降到合格範圍內，所剩油料就是成績。
        while True:
            if round(state.vspeed,0) >= -40: # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小。
                # 達標了
                score = state.fuel
                break
            else: # 未達標，繼續噴火把負的 vspeed 速度降到合格範圍內
                state.next(0,1)
    cost = fuel0 - score
    return cost 

%f state_list :> ["自由落體"]  cost_of_state :> (pop()) -->
%f state_list :> ["飛向太空"]  cost_of_state :> (pop()) -->
%f state_list :> ["安全降落"]  cost_of_state :> (pop()) -->
%f state_list :> ["更成功"]   cost_of_state :> (pop()) --> cr
        

state_list :> ["自由落體"]  cost_of_state :> (pop()) --> 1366 (<class 'int'>)
state_list :> ["飛向太空"]  cost_of_state :> (pop()) --> 1934 (<class 'int'>)
state_list :> ["安全降落"]  cost_of_state :> (pop()) --> 322 (<class 'int'>)
state_list :> ["更成功"]   cost_of_state :> (pop()) --> 288 (<class 'int'>)



In [7]:
# 模擬試跑 genome. Genome 可能太長或不夠，傳回訂正好的 genome 以及最後 state  
def simulate_of_genome(input_genome: GENOME) -> Tuple[GENOME,STATE]:
    genome = input_genome[:]  # make a copy to isolate the original
    state = State(**state0.__dict__)  # init state for simulation
    # 照 genome 跑到觸地、飛走、或油料耗盡時為止， genome 可能含有無效的長度要 cut 掉以免浪費時間嘗試。
    for i in range(len(genome)): 
        state.next(*genome[i])  # run 1 step 
        if round(state.y,0) < 100 or state.y > 3000 or state.fuel <= 0:  # 這高度、油料不能再繼續了
            genome = genome[:i+1]  # 太長的尾巴切掉
            break
        
    # 接續以上跑完 input genome 之後的 state 看看 input genome 是否太短需要補綴 genes 讓 lander 觸地。
    while round(state.y,0) >= 100 and state.y <= 3000 and state.fuel > 0:
        genome.append(genome[-1][:])  # 注意！若不用 [:] slice copy 則 append 進去的都是同一個。
        state.next(*genome[-1])  # 簡單照抄最後一 step
        
    return genome,state  # 這個 return genome 是 input genome 的訂正版

# 很妙，這產生「自由落體」的 genome 與最後 state 
genome, last_state = simulate_of_genome([[0,0]])
%f cr ( 「自由落體」 ) genome -->
%f last_state -->
%f last_state cost_of_state :> (pop()) -->
    
# 而這產生「全速飛走」的 genome 與最後 state 
genome, last_state = simulate_of_genome([[0,1]])
%f cr ( 「全速飛走」 ) genome -->
%f last_state -->
%f last_state cost_of_state :> (pop()) --> cr


cr ( 「自由落體」 ) genome --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]] (<class 'list'>)
last_state --> {'step': 36, 'x': 0, 'y': 95.2720000000003, 'hspeed': 0, 'vspeed': -133.59599999999998, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)
last_state cost_of_state :> (pop()) --> 1366 (<class 'int'>)

cr ( 「全速飛走」 ) genome --> [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 

In [110]:
# fitness function 
# https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)
def sa(initial: GENOME) -> Tuple[GENOME,int,int] :
    current = initial[:]
    current, current_last_state = simulate_of_genome(current)
    current_cost = cost_of_state(current_last_state)
    
    temp = 1.0  # (1/50)*(2*fuel0)  # <---- [ ] Hyperparameter, 來自 Einstein's problem 的心得，取總分的 1/15 
    num_iterations = 0

    i = 0
    while i < 2000:  # 若想安全降落為止，可能不如有限次數換個 epoch 試試運氣
        neighbour = get_random_neighbour(current)
        neighbour, neighbour_last_state = simulate_of_genome(neighbour)
        neighbour_cost = cost_of_state(neighbour_last_state)
        assert not (current is initial)
        assert not (current is neighbour)
        
        cost_delta = neighbour_cost - current_cost
        if cost_delta < 0:  # neighbour 較好，都接受。
            peforth.bp(7,locals())  # <--------- 
            current, current_cost = neighbour, neighbour_cost
            i = 0  # 有希望，繼續延長
            print({"cost":current_cost, "delta":cost_delta, "temperature":round(temp,2), "exp": "n/a"})
            if abs(round(neighbour_last_state.vspeed,0)) <= 40 and round(neighbour_last_state.y,0) < 100:
                print('B i n g o !')
                break;
        elif cost_delta > 0:  # 雖然 neighbour 不好，也偶而接受。
            exp = math.exp(-cost_delta/temp) 
            if random.random() < exp:
                current, current_cost = neighbour, neighbour_cost
                i = 0  # 有希望，繼續延長
                print({"cost":current_cost, "delta":cost_delta, "temperature":round(temp,2), "exp": round(exp,2)})
                peforth.bp(8,locals())  # <--------- 
        else:  # 排除相等的狀況，不要浪費時間、debug 版面 
            pass

        num_iterations += 1
        if num_iterations % 10 == 0 and temp > 0.10:  # <---- [ ] Hyperparameter
            temp -= 0.05  # <---- [ ] Hyperparameter

        i += 1
        
    return current, num_iterations, current_cost, neighbour_last_state

In [ ]:
def epoch(state=None):
    nationalities = [ 'dane',      'brit',   'swede',       'norwegian', 'german'    ]
    colours       = [ 'yellow',    'red',    'white',       'green',     'blue'      ]
    animals       = [ 'horse',     'cat',    'bird',        'fish',      'dog'       ]
    beverages     = [ 'water',     'tea',    'milk',        'coffee',    'root beer' ]
    cigars        = [ 'pall mall', 'prince', 'blue master', 'dunhill',   'blends'    ]

    random.shuffle(nationalities)
    random.shuffle(colours      )
    random.shuffle(animals      )
    random.shuffle(beverages    )
    random.shuffle(cigars       )
    
    attributes = [colours, nationalities, beverages, cigars, animals]

    NUM_HOUSES = 5
    initial = []
    s = '['
    
    if state: 
        initial = state; 
    else:
        for i in range(NUM_HOUSES):
            initial.append([attr[i] for attr in attributes])
    
    
    # random.seed(100)

    solution, iterations = sa(initial)

    for house in solution:
        s += str(house) + ',';
    s = s[:-1] + ']';
    return s,iterations

In [ ]:
if __name__ == "__main__":
    ss = '';
    for i in range(10000):
        s, iter = epoch()
        ss += "{} Number of iterations:{}\n".format(s, iter)
        print('.', end="")
    %f ss char results.txt writeTextFile 
    %f .( Done !!! ) cr

### SA tested with a given genome

In [9]:
%f bd* be 6
%f sample_genomes :> ["安全降落"]
random.seed(100)
genome, iterations, cost, state = sa(peforth.pop())

%f \ genome count --> ==>
%f iterations -->
%f cost -->
%f state -->
cost_of_state(state)

B i n g o !
6>6>

 neighbour_last_state . cr


{'step': 91, 'x': 0, 'y': 97.6045000000063, 'hspeed': 0, 'vspeed': -15.700999999999876, 'fuel': 178, 'angle': 0, 'power': 4, 'force': 0.28900000000000015}
6>

 exit


iterations --> 1 (<class 'int'>)
cost --> 322 (<class 'int'>)
state --> {'step': 91, 'x': 0, 'y': 97.6045000000063, 'hspeed': 0, 'vspeed': -15.700999999999876, 'fuel': 178, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)


322

In [ ]:
%f genome -->
genome , state = simulate_of_genome(genome)
%f genome -->
%f state -->
cost_of_state(state)

In [19]:
%f sample_genomes :> ["安全降落"] count nip -->

sample_genomes :> ["安全降落"] count nip --> 91 (<class 'int'>)


In [117]:
# 擾動【安全降落】用來測試 sa() 的容忍度，也就是【局部最佳】的範圍。
# 發現 'y': -1180.6079999999897 也 bingo <--- 有問題
%f bd*
random.seed(0)
_ = sample_genomes["安全降落"][:-11]
_.append([0,-1])
genome, iterations, cost, state = sa(_)
%f \ genome count --> ==>
%f iterations -->
%f cost -->
%f state -->
cost_of_state(state)

{'cost': 442, 'delta': -15, 'temperature': 1.0, 'exp': 'n/a'}
B i n g o !
iterations --> 3 (<class 'int'>)
cost --> 442 (<class 'int'>)
state --> {'step': 130, 'x': 0, 'y': -4019.949999999992, 'hspeed': 0, 'vspeed': -40.429999999999815, 'fuel': 58, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)


442